Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [3]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import numpy as np
from pymystem3 import Mystem
m = Mystem()
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Почистим текст от лишних символов, отдельных букв и привдём все к одному регистру

In [5]:
def cleaning(row):
    row = re.sub(r"(?:\n|\r)", " ", row)
    row = re.sub(r"[^a-zA-Z ]+", "", row).strip()
    row = row.lower()
    return row

df['text'] = df['text'].apply(cleaning)
df.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
3,more i cant make any real suggestions on impro...,0
4,you sir are my hero any chance you remember wh...,0


In [6]:
corpus = df['text'].values
def lemmatize(text):
    lemm = m.lemmatize(text)
    return "".join(lemm)

corpus[0] = lemmatize(corpus[0])
corpus

array(['explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired now\n',
       'daww he matches this background colour im seemingly stuck with thanks  talk  january   utc',
       'hey man im really not trying to edit war its just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info',
       ...,
       'spitzer   umm theres no actual article for prostitution ring   crunch captain',
       'and it looks like it was actually you who put on the speedy to have the first version deleted now that i look at it',
       'and  i really dont think you understand  i came here and my idea was bad right away  what kind of community goes you have bad ideas go away instead of helping rewrite th

# 2. Обучение

In [7]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)
target=df['toxic'].values
f_other, f_test, t_other, t_test = train_test_split(tf_idf, target, test_size = .1, random_state = 1)
f_train, f_valid, t_train, t_valid = train_test_split(f_other, t_other, shuffle=False, test_size=0.25, random_state = 1)

f_train.shape[0], f_valid.shape[0], f_test.shape[0]

(107709, 35904, 15958)

In [8]:
%%time
pipe = Pipeline([
    (
    ('model', LogisticRegression(random_state=1, solver='liblinear', max_iter=100))
    )
    ])


param_grid = [
        {
#            'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'model': [LogisticRegression(random_state=1, solver='liblinear')],
            'model__penalty' : ['l1', 'l2'],
            'model__C': list(range(1,15,3))
        }
]
grid = GridSearchCV(pipe, param_grid=param_grid, scoring='f1', cv=3, verbose=True)
best_grid = grid.fit(f_train, t_train)
print('Best parameters is:', grid.best_params_)
print('Best score is:', grid.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.5min finished


Best parameters is: {'model': LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=1, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False), 'model__C': 4, 'model__penalty': 'l1'}
Best score is: 0.7682109973074683
CPU times: user 1min 50s, sys: 1min 43s, total: 3min 34s
Wall time: 3min 35s


In [9]:
model = LogisticRegression(random_state=1, C = 4, penalty = 'l1', solver='liblinear', max_iter=100)
model.fit(f_train, t_train)
valid_pred = model.predict(f_valid)
f1_score(t_valid, valid_pred)


0.7683949084135362

f1 ожидаемо ниже, чем на трейне, но в целом норм.

# 3. Выводы

In [10]:
model = LogisticRegression(random_state=1, C = 4, penalty = 'l1', solver='liblinear', max_iter=100)
model.fit(f_train, t_train)
test_pred = model.predict(f_test)
f1_score(t_test, test_pred)

0.7787787787787788

F1 выше 0.75 - модель обучилась с качеством выше требуемого.